<a href="https://colab.research.google.com/github/yadavrishikesh/BayesNF/blob/main/Portland_HourlyTraffic_DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Install necessary packages
!pip install -q bayesnf cartopy contextily geopandas

In [9]:
import warnings
warnings.simplefilter('ignore')

import contextily as ctx
import geopandas as gpd
import jax
import numpy as np
import pandas as pd
import os
import datetime
from urllib.parse import urlparse
from bayesnf.spatiotemporal import BayesianNeuralFieldMAP

In [10]:
def run_portland_model(train_url, test_url):
    """
    Trains a Bayesian Neural Field MAP model on specified Portland train data and tests on test data.

    Parameters:
    - train_url (str): URL to the training dataset CSV file.
    - test_url (str): URL to the test dataset CSV file.

    Returns:
    - predictions_df (DataFrame): DataFrame containing true values, predicted mean, and quantiles.
    """
    # Determine base directory from train_url and create unique outputs directory
    train_name = os.path.basename(urlparse(train_url).path).split('.')[0]
    test_name = os.path.basename(urlparse(test_url).path).split('.')[0]
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = os.path.join(os.getcwd(), 'outputs', f'{train_name}_{test_name}_{timestamp}')
    os.makedirs(output_dir, exist_ok=True)

    # Download and load datasets
    !wget -q {train_url} -O train_data.csv
    !wget -q {test_url} -O test_data.csv
    df_train = pd.read_csv('train_data.csv', index_col=0, parse_dates=['datetime'])
    df_test = pd.read_csv('test_data.csv', index_col=0, parse_dates=['datetime'])

    # Log-transform response in training data
    df_train_log = df_train.copy()
    df_train_log['response'] = np.log1p(df_train_log['response'])

    # Define and train the Bayesian Neural Field model
    model = BayesianNeuralFieldMAP(
        width=512,
        depth=2,
        freq='H',
        seasonality_periods=['D'],
        num_seasonal_harmonics=[2],
        feature_cols=['datetime', 'lat', 'lon', 'speed', 'occupancy'],
        target_col='response',
        observation_model='NORMAL',
        timetype='index',
        standardize=['lat', 'lon', 'speed', 'occupancy'],
    )

    model = model.fit(df_train_log,
                      seed=jax.random.PRNGKey(0),
                      ensemble_size=32,
                      num_epochs=2500)

    # Process test data
    df_test_cleaned = df_test.dropna(subset=['response'])
    df_test_cleaned_log = df_test_cleaned.copy()
    df_test_cleaned_log['response'] = np.log1p(df_test_cleaned_log['response'])

    # Make predictions
    yhat, yhat_quantiles = model.predict(df_test_cleaned, quantiles=(0.025, 0.5, 0.975))

    # Compile results into DataFrame
    predictions_df = pd.DataFrame({
        'datetime': df_test_cleaned.index,
        'True_Values': df_test_cleaned['response'],
        'Predicted_Mean': yhat_quantiles[1],
        'Quantile_0.025': yhat_quantiles[0],
        'Quantile_0.975': yhat_quantiles[2]
    })

    # Save output in the unique outputs directory
    output_path = os.path.join(output_dir, 'predictions_output.csv')
    predictions_df.to_csv(output_path, index=False)

    return predictions_df

In [11]:
# Spatial predictions
train_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/train_data_Portland_hourly_pred-type_spatIntpl.csv'
test_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/test_data_Portland_hourly_pred-type_spatIntpl.csv'
predictions_df = run_portland_model(train_data_url, test_data_url)
predictions_df.head()


,datetime,True_Values,Predicted_Mean,Quantile_0.025,Quantile_0.975
1,1,7.001246,2.139726,2.067398,2.209277
2,2,6.966024,2.129271,2.057316,2.198401
3,3,6.659294,2.108565,2.037830,2.176650
4,4,6.538140,2.089637,2.020095,2.158311
5,5,6.505784,2.085215,2.015083,2.153149


In [12]:
# Forecasting
train_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/train_data_Portland_hourly_pred-type_forecast.csv'
test_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/test_data_Portland_hourly_pred-type_forecast.csv'
predictions_df = run_portland_model(train_data_url, test_data_url)
predictions_df.head()

,datetime,True_Values,Predicted_Mean,Quantile_0.025,Quantile_0.975
1,1,5.726848,1.886092,1.822022,1.950341
2,2,5.198497,1.809259,1.744622,1.877539
3,3,5.278115,1.784745,1.719229,1.859140
4,4,5.616771,1.829296,1.763493,1.903974
5,5,6.356108,1.942014,1.876632,2.012542


In [13]:
# Space-time predictions
train_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/train_data_Portland_hourly_pred-type_spaceTime.csv'
test_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/test_data_Portland_hourly_pred-type_spaceTime.csv'
predictions_df = run_portland_model(train_data_url, test_data_url)
predictions_df.head()

,datetime,True_Values,Predicted_Mean,Quantile_0.025,Quantile_0.975
1,1,5.899897,2.010926,1.944469,2.079808
2,2,5.683580,1.948411,1.883173,2.018611
3,3,5.783825,1.948231,1.882609,2.021145
4,4,6.331502,2.008454,1.942338,2.078874
5,5,7.037028,2.088498,2.019425,2.157726


In [16]:
# Mount Google Drive to access it from Colab
#from google.colab import drive
#drive.mount('/content/drive')

# Copy the 'outputs' directory to Google Drive
!cp -r /content/outputs /content/drive/MyDrive/BayesNF/outputs_Portland_hourly
